<a href="https://colab.research.google.com/github/ChristianKitte/SparkProjekt/blob/main/notebook/Wordcount_mit_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung des Notebooks

In [1]:
# Installation  von Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

print("Java ist installiert...")

# Download und Entpacken von Spark (Versionsnummer anpassen!)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

print("Spark ist verfügbar...")

# Setzen der Systemvariablen für Java und Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

print("Umgebungsvariablen sind gesetzt...")

Java ist installiert...
Spark ist verfügbar...
Umgebungsvariablen sind gesetzt...


In [2]:
!pip install findspark
print("FindSpark wurde installiert...")

!pip install pyspark
print("PySpark wurde installiert...")

FindSpark wurde installiert...
PySpark wurde installiert...


In [3]:
try: 
  import findspark
  from pyspark import SparkContext, SparkConf
  
  findspark.init()
  
  print("FindSpark und PySpark wurden initialisiert")
except ImportError: 
  raise ImportError("Fehler bei der Initialiserung von FindSpark und PySpark")

FindSpark und PySpark wurden initialisiert


# Einlesen der Textdatei

In [4]:
import requests 

def get_file_from_url(file_url, place_to_save):
  req = requests.get(file_url, stream = True) 

  with open(place_to_save, "wb") as file: 
	  for block in req.iter_content(chunk_size = 1024): 
		  if block: 
			  file.write(block) 
     
  print("Die Datei wurde angelegt: ", file_url)

print("Die Funktion get_file_from_url wurde angelegt...")


Die Funktion get_file_from_url wurde angelegt...


In [9]:
def cut_file(start, end, file, backup):
  try:
    with open(file, "r") as source:
      lines = source.readlines()
    
    source.close()
  
    start_count = start
    end_count = end

    print("Start: ", start_count)
    print("Ende: ", end_count)

    current_count = 0
  
    with open(backup, "w") as target:
      for line in lines:
        if current_count >= start_count and current_count <= end_count:
          target.write(line)

        current_count = current_count + 1   
    
    target.close()

    print("Fertig...")

  except ValueError:
    print("Fehler",ValueError)

print("Die Funktion cut_file wurde angelegt...")

Die Funktion cut_file wurde angelegt...


In [5]:
file_url = "https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt"
place_to_save = "/content/shakespeare.txt"

In [6]:
# Datei von der Quelle nach Colab laden

file_url = "https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt"
place_to_save = "/content/shakespeare.txt"

get_file_from_url(file_url, place_to_save)

print("Datei wurde geladen: ", file_url)

Die Datei wurde angelegt:  https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt
Datei wurde geladen:  https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt


In [10]:
# Unnötige Zeilen am Ende und am Start entfernen

file_source = "/content/shakespeare.txt"
file_target = "/content/shakespeare_neu.txt"

cut_file(244,124438,file_source, file_target)

print("Datei wurde beschnitten...")

Start:  244
Ende:  124438
Fertig...
Datei wurde beschnitten...


# Auszählen der Wörter

In [52]:
from pyspark import SparkContext 
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext 
from pyspark.sql import DataFrameReader


#sc = SparkContext("local[3]","word_count")
#sc.setLogLevel("ERROR")

#from pyspark.sql import SparkSession
#spark=SparkSession.builder.appName('Basics').getOrCreate()

#Version 1
#spark=SparkSession.builder.appName('WordCounter').getOrCreate()
#sc = spark.sparkContext()

#Version 2
#sc = SparkContext("local[3]","WordCounter")
#sc.setLogLevel("ERROR")

lines=sc.textFile(file_target).map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower()).filter(lambda linex: linex.strip() != "") 
for line in lines.collect()[0:30]:
  print(line) 

#print("Zeilen: " + str(lines.count()))

words=lines.flatMap(lambda line: line.split(" ")) \
  .map(lambda word: (word, 1)) \
  .reduceByKey(lambda a,b:a+b)

#for x in words[0:20]:
#  print(x)

#print(words.first)
sorted_counts = words.sortBy(lambda wordCounts: wordCounts[1], ascending=False)

#print("Wörter: " + str(words.count()))

#y=words.sortBy(lambda wc:wc[1], ascending=False)

#wordCounts=words.countByValue()
#print("Anzahl der unterschiedlichen Wörter: "+ str(len(wordCounts.items())))

#for word, count in words.collect()[0:10]:
#  print("Wort: {} - Anzahl: {}".format(word, count)) 

i = 0
for word, count in sorted_counts.collect()[0:30]:
    print("{} : {} : {} ".format(i, word, count))
    i += 1

1609
the sonnets
by william shakespeare
                     1
  from fairest creatures we desire increase 
  that thereby beauty's rose might never die 
  but as the riper should by time decease 
  his tender heir might bear his memory:
  but thou contracted to thine own bright eyes 
  feed'st thy light's flame with self substantial fuel 
  making a famine where abundance lies 
  thy self thy foe  to thy sweet self too cruel:
  thou that art now the world's fresh ornament 
  and only herald to the gaudy spring 
  within thine own bud buriest thy content 
  and tender churl mak'st waste in niggarding:
    pity the world  or else this glutton be 
    to eat the world's due  by the grave and thee 
                     2
  when forty winters shall besiege thy brow 
  and dig deep trenches in thy beauty's field 
  thy youth's proud livery so gazed on now 
  will be a tattered weed of small worth held:  
  then being asked  where all thy beauty lies 
  where all the treasure of thy lusty da

In [ ]:
print("...hier kommt der weitere Code")

...hier kommt der weitere Code
